### Initialization

In [401]:
import glob
import json
import re
from nltk import tokenize
import textdistance
import random
import string
import os
import numpy
import pandas
import fasttext
import gensim


import editdistance

In [2]:
from utils.text_extraction import *
from utils.document_processing import *

In [6]:
# import nltk
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("udpipe/udpipe-ud-2.4-190531/czech-pdt-ud-2.4-190531.udpipe")

In [5]:
def randomTokens():
    """Generate a random string of fixed length """
    tokensLength = int(random.gauss(4,2))+1
    letters = string.ascii_lowercase
    tokens = []
    for _ in range(tokensLength):
        stringLength = int(random.gauss(6,3)+1)
        tokens.append(''.join(random.choice(letters) for i in range(stringLength)))
    return tokens

### Subject extraction

In [6]:
class SubjectExtractor:
    
    def extract(self):
        return ['ahoj jak se mas']

class RandomSubjectExtractor(SubjectExtractor):
    
    def extract(self):
        subjects_num = int(numpy.random.exponential())+1
        return [randomTokens() for _ in range(subjects_num)]
    
class ReferenceSubjectExtractor(SubjectExtractor):
    
    _REF_FILENAME = '_ref.json'
    _path = None
    _ref = None
    _ref_subjects = None
    
    def __init__(self, path):
        self._path = path
    
    def extract(self):
        filename = os.path.join(self._path, self._REF_FILENAME)
        with open(filename, encoding='utf-8') as f:
            data = f.read()
            self._ref = json.loads(data)
            self._ref_subjects = [item['name'] for item in self._ref['subject']['items']]
        return self._ref_subjects

In [1020]:
class SubjectContextExtractor():
    
    _keywords = None
    _subj_range = None
    
    def __init__(self, keywords={"Předmět smlouvy":1}, subj_range=2000):
        self._keywords = keywords
        self._subj_range = subj_range
        
    def get_all_occurrences(self, text):
        occurrences = []
        for keyword in self._keywords:
            occ = find_all_occurences_in_string(keyword, text)
            for o in occ:
                rat = self._keywords[keyword]
                koef = 1
                matched = keyword.lower()
                # Exact pattern match
                if text[o:min(o+len(keyword),len(text))] == keyword:
                    koef+=1.5
                    matched = keyword
                # Upper case pattern match
                if text[o:min(o+len(keyword),len(text))] == keyword.upper():
                    koef+=1.5
                    matched = keyword.upper()
                # Nearly linebreak after the pattern (chapter title)
                if '\n' in text[o:min(o+len(keyword)*3,len(text))]:
                    koef+=2
                # Newline followed by a number preceding the pattern (chapter numbering)
                if re.search(r"\n[ ]*[0-9]", text[max(o-20,0):o]):
                    koef+=2
                # Nearly verb ' je ' after the pattern (subject sentence matching)
                if ' je ' in text[o:min(o+len(keyword)*2,len(text))]:
                    koef+=2
                # Word 'článek' preceding the pattern (chapter header)
                if 'článek' in text[max(o-20,0):o].lower():
                    koef+=2
                # Chars 'I' preceding the pattern (chapter numbering)
                if text[max(o-20,0):o].count('I')>1:
                    koef+=2
                rat*=koef
                occurrences.append({'keyword':matched, 'rat': rat, 'occ':o})
        return occurrences
    
    def get_subject_context_old(self, text):
        occurrences = self.get_all_occurrences(text)
        df = pandas.DataFrame(occurrences).T
        df = df.sort_values('rat')
        df = flatten_column(df, 'occ')
        if len(df.index)>0:
            occ = df.iloc[0]
        else:
            return None
        start = occ['occ_flat'] + len(occ.name)
        end = min(start + self._subj_range, len(text))
        subj_context = text[start:end]
        return subj_context

    def get_subject_context(self, text):
        bin_size = int(self._subj_range/3)
        bins = int(len(text)/bin_size)+1
        text = text.rjust(bins*bin_size)

        occurrences = self.get_all_occurrences(text)
        df = pandas.DataFrame(occurrences, columns=['keyword', 'rat', 'occ'])        
        
        hist = numpy.histogram(df['occ'], bins, weights=df['rat'], range=(0,len(text)))
        
        score = numpy.convolve(hist[0], numpy.array([1,1,2,-2,-1,-1]))[2:-3]
        subj_bin = numpy.argmax(score)
        
        start = subj_bin * bin_size
        end = min(start + self._subj_range, len(text))
        subj_context = text[start:end]
        return subj_context

    
    def show_occurrence_distribution(self, text):
        bin_size = int(self._subj_range/3)
        bins = int(len(text)/bin_size)+1
        text = text.rjust(bins*bin_size)

        occurrences = self.get_all_occurrences(text)
        df = pandas.DataFrame(occurrences)
        hist = numpy.histogram(df['occ'], bins, weights=df['rat'], range=(0,len(text)))
        pandas.DataFrame(hist[0]).plot.bar()

### EvaluationMachine

In [7]:
class EvaluationMachine:
    
    _core = None
    _subjects = None
    _ref_subjects = None
    _similarity_matrix = None
    
    def __init__(self, core=None):
        self._core = core
    
    def evaluateSubjects(self, subjects, ref_subjects):
        self._subjects = [' '.join(s) if isinstance(s, list) else s for s in subjects]
        self._ref_subjects = [' '.join(s) if isinstance(s, list) else s for s in ref_subjects]
        return self.evaluate()
    
    def evaluate(self):
        self.compute_matrix()
        return self._similarity_matrix
        
    def compute_matrix(self):
        sl = len(self._subjects)
        rl = len(self._ref_subjects)
        result_matrix = numpy.zeros((sl, rl))
        for i in range(sl):
            for j in range(rl):
                d = self._core.compute(self._subjects[i], self._ref_subjects[j])
                result_matrix[i][j] = d
        self._similarity_matrix = result_matrix
        
    def display(self, with_headers=False):
        if with_headers:
            df = pandas.DataFrame(self._similarity_matrix, columns=self._ref_subjects, index=self._subjects)
        else:
            df = pandas.DataFrame(self._similarity_matrix)
        display(df)
        
    
class RandomEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(RandomSimilarityMachine())
        
class JaccardEvaluationMachine(EvaluationMachine):

    def __init__(self, core=None):
        super().__init__(JaccardSimilarityMachine())
        
class FastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(FastTextSimilarityMachine(core))
        
class GensimFastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(GensimFastTextSimilarityMachine(core))

In [937]:
class SubjectContextSelectionEvaluationMachine():
    
    _contracts = None
    _df_contracts = None
    _subj_context_extractor = None
    _func_param = None
    
    def __init__(self, contracts, subj_context_extractor=SubjectContextExtractor, func_param=None):
        self._contracts = contracts
        self._subj_context_extractor = subj_context_extractor
        self._func_param = func_param
    
    def process(self):
        for contr in self._contracts:
            contr_text = contr['text']
            contr['subj_context'] = self._subj_context_extractor(self._func_param).get_subject_context(contr_text)
        self._df_contracts = pandas.DataFrame(self._contracts).set_index('id')
        return self._df_contracts
    
    def evaluate(self):
        return self._df_contracts['subj_context'].isna().sum()

In [935]:
class ComplexSubjectContextSelectionEvaluationMachine(SubjectContextSelectionEvaluationMachine):
    
    _path_to_data = None
    
    def __init__(self, path_to_data, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._path_to_data = path_to_data
    
    def process(self):
        super().process()
        self._df_contracts = validate_subj_contexts(self._df_contracts, self._path_to_data)
        self._df_contracts['valid']=False
        df_tmp = self._df_contracts[self._df_contracts.valid_rat>0.5].copy()
        df_tmp['valid']=True
        self._df_contracts['valid']=df_tmp['valid']
        return self._df_contracts
        
    def evaluate(self):
        return self._df_contracts['valid_rat'].mean()*100

### Similarity functions

In [9]:
class SimilarityMachine:
    
    _text = None
    _reference = None
    _model = None
    
    def __init__(self, model=None):
        self._model = model
        
    def init_case(self, text, reference):
        self._text = text
        self._reference = reference
        
    def preprocess(self):
        None
    
    def compute(self, text, reference):
        self.init_case(text, reference)
        self.preprocess()
        return self._inner_compute()
    
    def _inner_compute(self):
        return 0.5
    
class RandomSimilarityMachine(SimilarityMachine):
    
    def _inner_compute(self):
        return random.random()
    
class JaccardSimilarityMachine(SimilarityMachine):
    
    def preprocess(self):
        if isinstance(self._text, str):
            self._text = self._text.split()
        if isinstance(self._reference, str):
            self._reference = self._reference.split()        

    def _inner_compute(self):
        return textdistance.jaccard(self._text, self._reference)

class SorensenDiceSimilarityMachine(JaccardSimilarityMachine):
    
    def _inner_compute(self):
        return textdistance.sorensen_dice(self._text, self._reference)

class FastTextSimilarityMachine(SimilarityMachine):
    
    def __init__(self, model=None):
        if model is None:
            path = 'fasttext/cc.cs.300.bin'
            print('Loading fasttext model from ' + path)
            model = fasttext.load_model(path)
        super().__init__(model)
    
    def _inner_compute(self):
        return 1/(numpy.linalg.norm(self._model[self._text] - self._model[self._reference])+1)
    
class GensimFastTextSimilarityMachine(SimilarityMachine):
    
    def __init__(self, model=None):
        if model is None:
            path = 'fasttext/cc.cs.300.bin'
            print('Loading fasttext model from ' + path)
            model = gensim.models.FastText.load_fasttext_format(path)
        super().__init__(model)
    
    def _inner_compute(self):
        return 1/(self._model.wmdistance(self._text, self._reference)+1)

### Documents 

In [11]:
import numpy
from utils.document_processing import *

class DatabaseDocumentLoader():
    
    _connection = None
    _raw_data = None
    _documents = None
    
    def __init__(self, connection):
        self._connection = connection
        
    def load_documents(self, query=None):
        cur = self._connection.cursor()
        if query is None:
            query = "select * from document where processed=True"
        print("Running query: " + query)
        cur.execute(query)
        self._raw_data = cur.fetchall()
        cur.close()
        return self._raw_data
    
    def prepare_documents(self, parts=10):
        documents = []
        total_docs = len(self._raw_data)
        print("Preparing total " + str(total_docs) + " documents")
        for i, doc in enumerate(self._raw_data):
            if i % (int(total_docs/parts)) == 0:
                print("Progress: {}%".format(numpy.ceil(i*100/total_docs)))
            doc_id = doc[0]
            contr_id = doc[1]
            doc_url = doc[2]
            doc_text = doc[8]
            doc_vec = count_occurence_vector(doc_text)
            documents.append({'id': doc_id, 'contr_id': contr_id, 'url': doc_url, 'text': doc_text, 'vector': doc_vec})
        self._documents = documents
        return self._documents

In [18]:
import numpy
import glob
from utils.text_extraction import *
from utils.document_processing import *

class ReferenceDocumentLoader():
    
    _path_to_docs = None
    _documents = None
    
    def __init__(self, path_do_docs):
        self._path_to_docs = path_do_docs
    
    def extract_text_from_documents(self, min_file_size=0, max_file_size=5242880):
        docs_paths = glob.glob(self._path_to_docs)
        filtered_docs_paths = [path for path in docs_paths \
                if not path.endswith('_ref.json') \
                and os.stat(path).st_size > min_file_size \
                and os.stat(path).st_size < max_file_size]
        extractionMachine = ExtractionMachine(save=True, filter_extracted=True)
        total_docs = len(filtered_docs_paths)
        print("Running extraction on total " + str(total_docs) + " documents")
        extractionMachine.extractFromDirs(filtered_docs_paths)
        extractions = extractionMachine._extractions
        self._documents = {extract:{'text': extractions[extract]} for extract in extractions}
        return self._documents
    
    def load_documents_from_extracts(self, ignore_names=[]):
        docs_paths = glob.glob(self._path_to_docs)
        extracts_paths = []
        for path in docs_paths:
            if not path.endswith('_ext'):
                continue
            if 'test_src' in path:
                continue
            ignore = False
            for contr_name in ignore_names:
                if contr_name in path:
                    ignore = True
                    break
            if ignore:
                continue
            extracts_paths.append(path)

        documents = {}
        total_extracts = len(extracts_paths)
        print("Loading total " + str(total_extracts) + " extracts")
        for path in extracts_paths:
            with open(path, 'r', encoding='utf-8') as f:
                text = f.read()
                key = path[:-4]
                documents[key] = {'text': text}
        self._documents = documents
        return self._documents
    
    def prepare_documents(self, parts=10):
        total_docs = len(self._documents)
        print("Preparing total " + str(total_docs) + " documents")
        for i, doc in enumerate(self._documents):
            if i % (int(total_docs/parts)) == 0:
                print("Progress: {}%".format(numpy.ceil(i*100/total_docs)))
            doc_text = self._documents[doc]['text']
            self._documents[doc]['vector'] = count_occurence_vector(doc_text)
        return self._documents

In [805]:
import numpy
import pandas
from utils.document_processing import *

class DocumentMatcher():
    
    _ref_documents = None
    _documents = None
    _aggregated_documents = None
    
    def __init__(self, ref_documents, documents):
        self._ref_documents = ref_documents
        self._documents = documents
        
    def count_most_similar_documents(self, parts=10, ignore_mask=char_ignore_mask('')):
        print("Collecting document matrix")
        doc_matrix = numpy.array([doc['vector'] for doc in self._documents])
        masked_doc_matrix = doc_matrix * ignore_mask

        total_docs = len(self._ref_documents)
        print("Counting most similar documents for total " + str(total_docs) + " reference documents")
        for i, ref_doc in enumerate(self._ref_documents):
            if i % (int(total_docs/parts)) == 0:
                print("Progress: {}%".format(numpy.ceil(i*100/total_docs)))
            ref_doc_text = self._ref_documents[ref_doc]['text']
            ref_vec = self._ref_documents[ref_doc]['vector']
            masked_ref_vec = ref_vec * ignore_mask
            diff = numpy.abs(masked_doc_matrix-masked_ref_vec)
            self._ref_documents[ref_doc]['diff'] = diff
            diff_sum = diff.sum(axis=1)
            self._ref_documents[ref_doc]['diff_sum'] = diff_sum
            self._ref_documents[ref_doc]['link'] = numpy.argsort(diff_sum)
            self._ref_documents[ref_doc]['top_link'] = self._ref_documents[ref_doc]['link'][0]
            self._ref_documents[ref_doc]['top_diff'] = self._ref_documents[ref_doc]['diff_sum'][self._ref_documents[ref_doc]['top_link']]
        return self._ref_documents

    def get_doc_for_file(self, file):
        for ref in self._ref_documents:
            if file in ref:
                most_similar = self._ref_documents[ref]['top_link']
                return self._documents[most_similar]
    
    def aggregate_documents(self):
        aggregated_documents = []
        for ref in self._ref_documents:
            ref_doc = self._ref_documents[ref]
            db_doc = self._documents[ref_doc['top_link']]
            path_parts = ref.split('\\')

            doc_id = db_doc['id']
            doc_name = path_parts[-1]
            contr_id = db_doc['contr_id']
            contr_name = '/'.join(path_parts[1:4])
            doc_text = db_doc['text']
            ref_text = ref_doc['text']
            doc_url = db_doc['url']
            doc_path = ref
            diff = ref_doc['top_diff']

            if ref_text.count('\n')/(len(ref_text)+1) > 0.25:
                continue

            agg_doc = { 'doc_id': doc_id, 'doc_name': doc_name, 'contr_id': contr_id, 'contr_name': contr_name,
                       'doc_text': doc_text, 'ref_text': ref_text, 'doc_url': doc_url, 'doc_path': doc_path, 'diff': diff }
            aggregated_documents.append(agg_doc)
        self._aggregated_documents = aggregated_documents
        return self._aggregated_documents
    
    def filter_aggregated(self, filter_ratio=0.1):
        df_aggregated = pandas.DataFrame(self._aggregated_documents)
        df_aggregated['ref_text_len'] = df_aggregated['ref_text'].str.len()
        df_aggregated['diff_ratio'] = df_aggregated['diff']/df_aggregated['ref_text_len']
        df_contract = df_aggregated[df_aggregated.diff_ratio < filter_ratio]
        return df_contract

In [12]:
loader = DatabaseDocumentLoader(psycopg2_conn)
loader.load_documents()
documents = loader.prepare_documents()

Running query: select * from document where processed=True
Preparing total 998 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


In [19]:
vzdirs = '../test-data/*/*/*/*'
ignore_contracts = ['2096','sportovni-areal-zs-slatina-1-etapa_25534',
                    'revitalizace-lokality-spaliste-k-u-stare-mesto-u-uherskeho-hradiste-akceptacni-cislo-12136894-cz-1-02-4-2-00-12-16399-dodatecne-prace_9496',
                    '35226', 'kompostujeme-v-obci-sebranice-dodavka-stepkovace-a-komposteru', 'zvysovani-povedomi-verejnosti-o-biodiverzite-luk-a-pastvin',
                    'nakup-noveho-uzitkoveho-elektromobilu-typu-bev-vozidlo-s-bateriovym-pohonem']

ref_loader = ReferenceDocumentLoader(vzdirs)
# ref_loader.extract_text_from_documents(0,1024)
ref_loader.load_documents_from_extracts(ignore_contracts)
ref_documents = ref_loader.prepare_documents()

Loading total 316 extracts
Preparing total 316 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


In [806]:
ignore_mask = count_occurence_vector('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNMáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ0123456789')
ignore_mask[0]=0
matcher = DocumentMatcher(ref_documents, documents)
matcher.count_most_similar_documents(ignore_mask=ignore_mask)
matcher.aggregate_documents()
df_contract = matcher.filter_aggregated()

Counting most similar documents for total 316 reference documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


In [27]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

### Measures

In [382]:
def save_valid_contexts(df_contracts, df_ref_documents):
    for index, row in df_contracts[df_contracts.valid==True].iterrows():
        path=df_ref_documents[df_ref_documents.contr_name==row.contr_name].iloc[0]['doc_path'].split('\\')[:-1]
        path.append('subj_context.txt')
        completed_path = '/'.join(path)
        with open(completed_path, 'w', encoding='utf8') as f:
            f.write(row.subj_context)

In [815]:
def validate_subj_contexts(df_contracts, vzdirs):
    df_contracts['valid_rat']=0
    df_contracts['ref_context']=None
    subj_context_paths = [path for path in glob.glob(vzdirs) if 'subj_context.txt' in path]
    for path in subj_context_paths:
        contr_name = '/'.join(path.split('\\')[1:4])
        row = df_contracts[df_contracts.contr_name==contr_name].iloc[0]
        with open(path, 'r', encoding='utf8') as f:
            ref_context = f.read()
        valid_rat = JaccardSimilarityMachine().compute(row['subj_context'], ref_context)
        df_contracts.valid_rat.loc[row.name] = valid_rat
        df_contracts.ref_context.loc[row.name] = ref_context        
    return df_contracts

In [808]:
contracts = []
for contr_name, docs in df_contract.groupby('contr_name'):
    contr_text = ''
    contr_ids = []
    for index, doc in docs.iterrows():
        doc_name = doc['doc_name']
        contr_text += '======================='+doc_name+'=======================\n'
        contr_text += doc['doc_text']
        contr_ids.append(doc['contr_id'])
    contr_id = get_most_frequent(contr_ids)
    contr = {'id': contr_id, 'contr_name': contr_name, 'text': contr_text}
    contracts.append(contr)

In [1009]:
scores2 = {}

In [1028]:
%%time 

vzdirs = '../test-data/*/*/*/*'

keywords = {
    'Předmět smlouvy':10,
    'Předmět díla':10,
    'Předmět plnění':10,
    'Předmět veřejné zakázky':10,
    'Vymezení předmětu':10,
    'Vymezení plnění': 10,
    'Název veřejné zakázky': 3,
    'Veřejná zakázka':1,
    'Veřejné zakázce':1,
    'Předmět': 1
   }
evaluator = ComplexSubjectContextSelectionEvaluationMachine(vzdirs, contracts, subj_context_extractor=SubjectContextExtractor, func_param=keywords)
evaluator.process()
score = evaluator.evaluate()
print('Score: ' + str(score))
print('Improve: ' + str(score - last_score))
last_score = score
# scores2['"Předmět"_1'] = score

Score: 97.5
Improve: 3.3968126599025936
Wall time: 813 ms


In [988]:
scores

{'base': 78.69533554251873,
 'exact_match': 85.06912091933681,
 'upper_case_match': 78.69533554251873,
 'chapter_title_newline_match': 84.1241836511366,
 'chapter_numbering_numeric_match': 82.26966607473528,
 'subject_sentence_match': 72.43105442376346,
 'chapter_header_match': 82.90891893654634,
 'chapter_numbering_alphabetic_match': 74.88185235697019,
 'all_match': 97.5,
 'w/o_exact_match': 91.79308165326701,
 'w/o_upper_case_match': 97.5,
 'w/o_chapter_title_newline_match': 94.67653977758769,
 'w/o_chapter_numbering_numeric_match': 87.36946331168959,
 'w/o_subject_sentence_match': 97.5,
 'w/o_chapter_header_match': 90.62725740925511,
 'w/o_chapter_numbering_alphabetic_match': 93.01077212261421}

In [1026]:
scores2

{'all': 97.5,
 '"Předmět smlouvy"_10': 75.02233664453941,
 '"Předmět díla"_10': 93.17233940556088,
 '"Předmět plnění"_10': 85.27830272948049,
 '"Předmět veřejné zakázky"_10': 89.35579291161477,
 '"Vymezení předmětu"_10': 91.61187160608195,
 '"Vymezení plnění"_10': 95.27538726333906,
 '"Název veřejné zakázky"_3': 86.90680098609205,
 '"Veřejná zakázka"_1': 95.30282872113969,
 '"Veřejné zakázce"_1': 96.41691394658753,
 '"Předmět"_1': 96.62181303116148}

### Playground

In [878]:
save_valid_contexts(df_contracts, df_contract)

In [182]:
decomposition = udp_pipeline.process(line[start:end])

In [183]:
print(decomposition)

# newdoc
# newpar
# sent_id = 1
# text = Předmětem smlouvy je provedení stavebně technických průzkumů, zpracování projektové dokumentace pro stavební povolení (dále jen „DSP“) včetně investorské činnosti (dále jen „IČ“) pro zajištění pravomocného stavebního povolení a pravomocného kolaudačního souhlasu, zpracování prováděcí dokumentace pro provedení stavby (dále jen „DPS“) dle $ 1 odst. 1 písm. f) vyhlášky 499/2006 Sb. (bude použita pro výběr dodavatele stavby) to vše na investiční akci „Projektová činnost pro rekonstrukci střešního pláště budovy Jiráskova 1320 Rychnov nad Kněžnou“, autorský dozor výše uvedené investiční akce a zajištění průkazu energetické náročnosti budovy v souladu s příslušnými ustanoveními zákona č. 183/2006 Sb., o územním plánování a stavebním řádu (stavební zákon), ve znění pozdějších předpisů (dále jen „stavební zákon“), včetně právních předpisů, které stavební zákon provádějí (veškeré výše uvedené společně dále jen jako „dílo“).
1	Předmětem	předmět	NOUN	NNIS7-